In [16]:
import pandas as pd

df = pd.read_csv("E:/Projects/Smart-Logistics-System/data/processed/dataset_with_delay_probability.csv")

print("Shape:", df.shape)
df.head()

Shape: (1000, 27)


,Latitude,Longitude,Inventory_Level,Temperature,Humidity,Precipitation(mm),Waiting_Time,User_Transaction_Amount,User_Purchase_Frequency,Asset_Utilization,...,Asset_ID_Truck_10,Asset_ID_Truck_2,Asset_ID_Truck_3,Asset_ID_Truck_4,Asset_ID_Truck_5,Asset_ID_Truck_6,Asset_ID_Truck_7,Asset_ID_Truck_8,Asset_ID_Truck_9,delay_probability
0,-65.7383,11.2497,390,27.0,67.8,10.023475,38,320,4,60.1,...,0,0,0,0,0,0,1,0,0,0.339254
1,22.2748,-131.7086,491,22.5,54.3,21.397599,16,439,7,80.9,...,0,0,0,0,0,1,0,0,0,0.870765
2,54.9232,79.5455,190,25.2,62.2,14.520535,34,355,3,99.2,...,1,0,0,0,0,0,0,0,0,0.452376
3,42.3900,-1.4788,330,25.4,52.3,2.613761,37,227,5,97.4,...,0,0,0,0,0,0,0,0,1,0.842010
4,-65.8477,47.9468,480,20.5,57.2,35.412586,56,197,6,71.6,...,0,0,0,0,0,0,1,0,0,0.471364


In [17]:
# ============================================
# RISK TIER CLASSIFICATION
# ============================================

def get_risk_level(prob):
    if prob > 0.85:
        return "Critical"
    elif prob > 0.70:
        return "High"
    elif prob >= 0.40:
        return "Medium"
    else:
        return "Low"

df["ml_risk_level"] = df["delay_probability"].apply(get_risk_level)

df[["delay_probability", "ml_risk_level"]].head()


,delay_probability,ml_risk_level
0,0.339254,Low
1,0.870765,Critical
2,0.452376,Medium
3,0.842010,High
4,0.471364,Medium


In [18]:
# ======================================
# WEATHER + TRAFFIC
# ======================================

def weather_traffic_override(row):

    # Heavy Traffic + Rain > 15mm → Critical
    if row["Precipitation(mm)"] > 15 and row["Traffic_Status_Heavy"] == 1:
        return "Critical"

    return row["ml_risk_level"]


# Apply override
df["final_risk_level"] = df.apply(weather_traffic_override, axis=1)

# Quick check
df[[
    "Precipitation(mm)",
    "Traffic_Status_Heavy",
    "ml_risk_level",
    "final_risk_level"
]].head()


,Precipitation(mm),Traffic_Status_Heavy,ml_risk_level,final_risk_level
0,10.023475,0,Low,Low
1,21.397599,1,Critical,Critical
2,14.520535,0,Medium,Medium
3,2.613761,1,High,High
4,35.412586,0,Medium,Medium


In [19]:
# =====================================
# RULE 2: ASSET STRESS
# =====================================

# Define risk priority order
risk_priority = {
    "Low": 1,
    "Medium": 2,
    "High": 3,
    "Critical": 4
}

def apply_asset_stress(row):
    if row["Asset_Utilization"] > 90:
        # If current risk is lower than High, escalate to High
        if risk_priority[row["ml_risk_level"]] < risk_priority["High"]:
            return "High"
    return row["ml_risk_level"]

df["final_risk_level"] = df.apply(apply_asset_stress, axis=1)

df[["Asset_Utilization", "ml_risk_level", "final_risk_level"]].head()

,Asset_Utilization,ml_risk_level,final_risk_level
0,60.1,Low,Low
1,80.9,Critical,Critical
2,99.2,Medium,High
3,97.4,High,High
4,71.6,Medium,Medium


In [20]:
# =====================================
# FINAL RISK DISTRIBUTION
# =====================================

print("ML Risk Distribution:")
print(df["ml_risk_level"].value_counts(normalize=True) * 100)

print("\nFinal Risk Distribution (After Overrides):")
print(df["final_risk_level"].value_counts(normalize=True) * 100)

ML Risk Distribution:
ml_risk_level
Medium      40.2
Low         27.1
Critical    23.9
High         8.8
Name: proportion, dtype: float64

Final Risk Distribution (After Overrides):
final_risk_level
Medium      32.0
High        24.3
Critical    23.9
Low         19.8
Name: proportion, dtype: float64


In [21]:
# =====================================
# SANITY CHECK 
# =====================================

weather_escalations = (
    ((df["Precipitation(mm)"] > 20) & (df["Traffic_Status_Heavy"] == 1)) |
    ((df["Precipitation(mm)"] > 15) & (df["Traffic_Status_Detour"] == 1))
).sum()

print("\nRows escalated by Weather Rule:")
print(weather_escalations)

print("\nRows escalated by Asset Rule:")
print((df["Asset_Utilization"] > 90).sum())



Rows escalated by Weather Rule:
132

Rows escalated by Asset Rule:
238


In [23]:
df.to_csv(
    "E:/Projects/Smart-Logistics-System/data/processed/dataset_with_risk_levels.csv",
    index=False
)

print("Risk Intelligence dataset saved successfully ✅")

Risk Intelligence dataset saved successfully ✅
